In [17]:
import os
from IPython.display import Markdown, HTML, display
from langchain_openai import ChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase


In [8]:
from sqlalchemy import create_engine
import pandas as pd


df = pd.read_excel("./employees_loc.xlsx")

In [118]:
database_file_path = "./db/test.db"

In [5]:
import os
from sqlalchemy import create_engine
import pandas as pd

# Ensure the directory exists
os.makedirs(os.path.dirname(database_file_path), exist_ok=True)

# Create an engine to connect to the SQLite database
engine = create_engine(f'sqlite:///{database_file_path}')

# Load the Excel file
file_url = "./employees_loc.xlsx"
df = pd.read_excel(file_url)

# Write the DataFrame to the SQLite database
df.to_sql(
    'all_states_history',
    con=engine,
    if_exists='replace',
    index=False
)

20000

In [56]:
MSSQL_AGENT_PREFIX = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [57]:
MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: 
-- Calculate distance using the Haversine formula
WITH Distance_Calc AS (
    SELECT 
        e.Employee_ID,
        e.Gates_ID AS Gate_ID,
        (6371 * acos(cos(radians(e.LATITUDE)) * cos(radians(e.Gate_LATITUDE)) 
        * cos(radians(e.Gate_LONGITUDE) - radians(e.LONGITUDE)) 
        + sin(radians(e.LATITUDE)) * sin(radians(e.Gate_LATITUDE)))) AS Distance_km
    FROM 
        all_states_history e

)

-- Convert distance to time in minutes (assuming an average walking speed of 5 km/h)
SELECT 
    Employee_ID,
    Gate_ID,
    Distance_km,
    (Distance_km / 5) * 60 AS Time_minutes
FROM 
    Distance_Calc
WHERE 
    Employee_ID = 'Employee_1' AND Gate_ID = 'gate_008';

Observation:
(0.224), (2)
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I used the Haversine formula to calculate the distance between each employee's location and a specific gate. First, I selected latitude and longitude coordinates for both employees and gates from the all_states_history table. The formula calculates the distance in kilometers by using these coordinates and Earth's radius (6371 km).

To convert the distance into time, I assumed an average walking speed of 5 km/h. The time in minutes is calculated by dividing the distance by 5 and then multiplying by 60. Finally, I filtered the results to get the distance and time specifically for Employee_1 and gate_008.
===> End of Example

"""

In [18]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()

········


In [20]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [21]:
db = SQLDatabase.from_uri(f'sqlite:///{database_file_path}')
toolkit = SQLDatabaseToolkit(db=db, llm=model)

In [21]:
QUESTION = """how much employee_1 is away from gate 005 in km and minutes
"""

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=model,
    toolkit=toolkit,
    top_k=30,
    verbose=True
    
    
)

In [22]:
agent_executor_SQL.invoke(QUESTION,handle_parsing_errors=True)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables

Action Input: 
all_states_historyI should query the schema of the table "all_states_history" to see the columns available for my query.
Action: sql_db_schema
Action Input: all_states_history
CREATE TABLE all_states_history (
	"Employee ID" BIGINT, 
	"Name" TEXT, 
	"Skill" TEXT, 
	"Shift" TEXT, 
	"Start Time" TEXT, 
	"End Time" TEXT, 
	"Gates_ID" TEXT, 
	"LONGITUDE" FLOAT, 
	"LATITUDE" FLOAT
)

/*
3 rows from all_states_history table:
Employee ID	Name	Skill	Shift	Start Time	End Time	Gates_ID	LONGITUDE	LATITUDE
1	Employee_1	Aircraft Painter	Morning	06:00	14:00	gate 007	-77.043	38.904
2	Employee_2	Aircraft Painter	Night	22:00	06:00	gate 004	-77.032	38.906
3	Employee_3	Aircraft Painter	Afternoon	14:00	22:00	gate 019	-77.023	38.903
*/I can now calculate the distance between Employee_1 and gate 005 using the latitude and longitude coordinates from the all_states_history table.
Action: sql_db_query
Action Input: 
SELECT 

{'input': 'how much employee_1 is away from gate 005 in km and minutes\n',
 'output': 'Employee_1 is approximately 1.15 km away from gate 005, and it would take around 13 minutes to walk there. \n\nExplanation: I calculated the distance between Employee_1 and gate 005 using the Haversine formula, considering the latitude and longitude coordinates. The query returned the distance in kilometers. To convert this distance to time in minutes, assuming an average walking speed of 5 km/h, I divided the distance by 5 and then multiplied by 60.'}

In [84]:
import numpy as np
from sqlalchemy import text

def get_available_employees_with_skill(skill_code):
    try:
        query = f"""
        SELECT Count("Employee ID")
        FROM all_states_history
        WHERE Skill LIKE '%{skill_code}%' 
        AND "Start Time" <= datetime('now') 
        AND "End Time" >= datetime('now');
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        if not result.empty:
            return result.to_dict('records')[0]
        else:
            return np.nan
    except Exception as e:
        print(e)
        return np.nan 

In [85]:
messages = [
    {"role": "user",
     "content": """ how many employees are available now who are Avionics Specialist?"""
    }
]

In [86]:
tools_sql = [
    {
        "type": "function",
        "function": {
            "name": "get_available_employees_with_skill",
            "description": """Retrieves the available employees with a certain skill.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "skill_code": {
                        "type": "string",
                        "description": """The skill (e.g., 'Avionics Specialist', 'Mechanical Systems Engineer')"""
                    }
                },
                "required": ["skill_code"]
            }
        }
    }
]


In [43]:
from openai import OpenAI
client = OpenAI()

In [87]:
import json

# Step 1: Inspect the objects
print(f"Model: {model}")
print(f"Messages: {messages}")
print(f"Tools: {tools_sql}")

# Step 2: Make the initial API call
response = client.chat.completions.create(
    model="gpt-4",  # Replace with your specific model string
    messages=messages,
    tools=tools_sql,
    tool_choice="auto",
)

# Step 3: Check the response
print(f"Response: {response}")

response_message = response.choices[0].message
print(f"Response Message: {response_message}")

tool_calls = response_message.tool_calls
print(f"Tool Calls: {tool_calls}")

if tool_calls:
    available_functions = {
        "get_available_employees_with_skill": get_available_employees_with_skill
    }
    
    # Process each tool call
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        print(f"Function Name: {function_name}")
        
        function_to_call = available_functions.get(function_name)
        function_args = json.loads(tool_call.function.arguments)
        
        print(f"Function Args: {function_args}")
        
        if function_to_call:
            function_response = function_to_call(
                skill_code=function_args.get("skill_code"),
            )
            print(f"Function Response: {function_response}")
            
            # Step 4: Append the tool's response to messages
            messages.append(
                {
                    "role": "assistant",
                    "content": function_response,
                    "tool_call_id": tool_call.id,
                }
            )
        else:
            print(f"Function {function_name} is not available.")
        
    # Print the updated messages to inspect the final structure
    print(f"Messages: {messages}")
    
    # Step 5: If necessary, you can make another API call here
else:
    print("No tool calls were made.")

Model: client=<openai.resources.chat.completions.Completions object at 0x000001D89B3EBCA0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D89900E9B0> root_client=<openai.OpenAI object at 0x000001D8989379A0> root_async_client=<openai.AsyncOpenAI object at 0x000001D89B3EBA90> openai_api_key=SecretStr('**********') openai_proxy=''
Messages: [{'role': 'user', 'content': ' how many employees are available now who are Avionics Specialist?'}]
Tools: [{'type': 'function', 'function': {'name': 'get_available_employees_with_skill', 'description': 'Retrieves the available employees with a certain skill.', 'parameters': {'type': 'object', 'properties': {'skill_code': {'type': 'string', 'description': "The skill (e.g., 'Avionics Specialist', 'Mechanical Systems Engineer')"}}, 'required': ['skill_code']}}}]
Response: ChatCompletion(id='chatcmpl-A1tWAiRalTR0RejeJZJZZmt5TQ7TI', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletio

In [88]:
"""SQLAlchemy wrapper around a database."""
from __future__ import annotations

from typing import Any, List, Optional

from sqlalchemy import MetaData, create_engine, inspect
from sqlalchemy.engine import Engine
from sqlalchemy.schema import CreateTable

from langchain import utils
from langchain import SQLDatabase as _SQLDatabase


class SQLDatabase(_SQLDatabase):
    """SQLAlchemy wrapper around a database."""

    def __init__(
        self,
        engine: Engine,
        schemas: List[str],
        metadata: Optional[MetaData] = None,
        ignore_tables: Optional[List[str]] = None,
        include_tables: Optional[List[str]] = None,
        sample_rows_in_table_info: int = 3,
        indexes_in_table_info: bool = False,
        custom_table_info: Optional[dict] = None,
        view_support: bool = False,
        max_string_length: int = 300,
    ):
        """Create engine from database URI."""
        self._engine = engine
        self._schema = None
        self._schemas = schemas
        if include_tables and ignore_tables:
            raise ValueError("Cannot specify both include_tables and ignore_tables")

        self._inspector = inspect(self._engine)

        # including view support by adding the views as well as tables to the all
        # tables list if view_support is True
        self._all_tables_per_schema = {}
        for schema in self._schemas:
            self._all_tables_per_schema[schema] = set(
                self._inspector.get_table_names(schema=schema)
                + (self._inspector.get_view_names(schema=schema) if view_support else [])
            )
        self._all_tables = set(f"{k}.{name}" for k, names in self._all_tables_per_schema.items() for name in names)

        self._include_tables = set(include_tables) if include_tables else set()
        if self._include_tables:
            missing_tables = self._include_tables - self._all_tables
            if missing_tables:
                raise ValueError(
                    f"include_tables {missing_tables} not found in database"
                )
        self._ignore_tables = set(ignore_tables) if ignore_tables else set()
        if self._ignore_tables:
            missing_tables = self._ignore_tables - self._all_tables
            if missing_tables:
                raise ValueError(
                    f"ignore_tables {missing_tables} not found in database"
                )
        usable_tables = self.get_usable_table_names()
        self._usable_tables = set(usable_tables) if usable_tables else self._all_tables

        if not isinstance(sample_rows_in_table_info, int):
            raise TypeError("sample_rows_in_table_info must be an integer")

        self._sample_rows_in_table_info = sample_rows_in_table_info
        self._indexes_in_table_info = indexes_in_table_info

        self._custom_table_info = custom_table_info
        if self._custom_table_info:
            if not isinstance(self._custom_table_info, dict):
                raise TypeError(
                    "table_info must be a dictionary with table names as keys and the "
                    "desired table info as values"
                )
            # only keep the tables that are also present in the database
            intersection = set(self._custom_table_info).intersection(self._all_tables)
            self._custom_table_info = dict(
                (table, self._custom_table_info[table])
                for table in self._custom_table_info
                if table in intersection
            )

        self._max_string_length = max_string_length

        self._metadata = metadata or MetaData()
        # including view support if view_support = true

        for schema in self._schemas:
            self._metadata.reflect(
                views=view_support,
                bind=self._engine,
                only=[v.split(".")[-1] for v in self._usable_tables if v.startswith(schema)],
                schema=schema,
            )

        # Add id to tables metadata
        for t in self._metadata.sorted_tables:
            t.id = f"{t.schema}.{t.name}"

    @classmethod
    def from_uri(
        cls,
        database_uri: str,
        schemas: Optional[List[str]] = None,
        engine_args: Optional[dict] = None,
        **kwargs: Any
    ) -> SQLDatabase:
        """Construct a SQLAlchemy engine from URI."""
        _engine_args = engine_args or {}
        engine = create_engine(database_uri, **_engine_args)
        return cls(engine, schemas=schemas, **kwargs)

    @classmethod
    def from_databricks(
        cls,
        catalog: str,
        schemas: Optional[List[str]] = None,
        host: Optional[str] = None,
        api_token: Optional[str] = None,
        warehouse_id: Optional[str] = None,
        cluster_id: Optional[str] = None,
        engine_args: Optional[dict] = None,
        **kwargs: Any,
    ) -> SQLDatabase:
        """
        Class method to create an SQLDatabase instance from a Databricks connection.
        This method requires the 'databricks-sql-connector' package. If not installed,
        it can be added using `pip install databricks-sql-connector`.

        Args:
            catalog (str): The catalog name in the Databricks database.
            schemas (List[str[): The schema name(s) in the catalog
            host (Optional[str]): The Databricks workspace hostname, excluding
                'https://' part. If not provided, it attempts to fetch from the
                environment variable 'DATABRICKS_HOST'. If still unavailable and if
                running in a Databricks notebook, it defaults to the current workspace
                hostname. Defaults to None.
            api_token (Optional[str]): The Databricks personal access token for
                accessing the Databricks SQL warehouse or the cluster. If not provided,
                it attempts to fetch from 'DATABRICKS_TOKEN'. If still unavailable
                and running in a Databricks notebook, a temporary token for the current
                user is generated. Defaults to None.
            warehouse_id (Optional[str]): The warehouse ID in the Databricks SQL. If
                provided, the method configures the connection to use this warehouse.
                Cannot be used with 'cluster_id'. Defaults to None.
            cluster_id (Optional[str]): The cluster ID in the Databricks Runtime. If
                provided, the method configures the connection to use this cluster.
                Cannot be used with 'warehouse_id'. If running in a Databricks notebook
                and both 'warehouse_id' and 'cluster_id' are None, it uses the ID of the
                cluster the notebook is attached to. Defaults to None.
            engine_args (Optional[dict]): The arguments to be used when connecting
                Databricks. Defaults to None.
            **kwargs (Any): Additional keyword arguments for the `from_uri` method.

        Returns:
            SQLDatabase: An instance of SQLDatabase configured with the provided
                Databricks connection details.

        Raises:
            ValueError: If 'databricks-sql-connector' is not found, or if both
                'warehouse_id' and 'cluster_id' are provided, or if neither
                'warehouse_id' nor 'cluster_id' are provided and it's not executing
                inside a Databricks notebook.
        """
        try:
            from databricks import sql  # noqa: F401
        except ImportError:
            raise ValueError(
                "databricks-sql-connector package not found, please install with"
                " `pip install databricks-sql-connector`"
            )
        context = None
        try:
            from dbruntime.databricks_repl_context import get_context

            context = get_context()
        except ImportError:
            pass

        default_host = context.browserHostName if context else None
        if host is None:
            host = utils.get_from_env("host", "DATABRICKS_HOST", default_host)

        default_api_token = context.apiToken if context else None
        if api_token is None:
            api_token = utils.get_from_env(
                "api_token", "DATABRICKS_TOKEN", default_api_token
            )

        if warehouse_id is None and cluster_id is None:
            if context:
                cluster_id = context.clusterId
            else:
                raise ValueError(
                    "Need to provide either 'warehouse_id' or 'cluster_id'."
                )

        if warehouse_id and cluster_id:
            raise ValueError("Can't have both 'warehouse_id' or 'cluster_id'.")

        if warehouse_id:
            http_path = f"/sql/1.0/warehouses/{warehouse_id}"
        else:
            http_path = f"/sql/protocolv1/o/0/{cluster_id}"

        uri = (
            f"databricks://token:{api_token}@{host}?"
            f"http_path={http_path}&catalog={catalog}"
        )
        return cls.from_uri(database_uri=uri, schemas=schemas, engine_args=engine_args, **kwargs)

    def get_table_info(self, table_names: Optional[List[str]] = None) -> str:
        """Get information about specified tables.

        Follows best practices as specified in: Rajkumar et al, 2022
        (https://arxiv.org/abs/2204.00498)

        If `sample_rows_in_table_info`, the specified number of sample rows will be
        appended to each table description. This can increase performance as
        demonstrated in the paper.
        """
        all_table_names = self.get_usable_table_names()
        if table_names is not None:
            missing_tables = set(table_names).difference(all_table_names)
            if missing_tables:
                raise ValueError(f"table_names {missing_tables} not found in database")
            all_table_names = table_names

        meta_tables = [
            tbl
            for tbl in self._metadata.sorted_tables
            if tbl.id in set(all_table_names)
            and not (self.dialect == "sqlite" and tbl.name.startswith("sqlite_"))
        ]

        tables = []
        for table in meta_tables:
            if self._custom_table_info and table.name in self._custom_table_info:
                tables.append(self._custom_table_info[table.name])
                continue

            # add create table command
            create_table = str(CreateTable(table).compile(self._engine))
            table_info = f"{create_table.rstrip()}"
            has_extra_info = (
                self._indexes_in_table_info or self._sample_rows_in_table_info
            )
            if has_extra_info:
                table_info += "\n\n/*"
            if self._indexes_in_table_info:
                table_info += f"\n{self._get_table_indexes(table)}\n"
            if self._sample_rows_in_table_info:
                table_info += f"\n{self._get_sample_rows(table)}\n"
            if has_extra_info:
                table_info += "*/"
            tables.append(table_info)
        final_str = "\n\n".join(tables)
        return final_str

In [13]:
file_url = "./Task Allocation Input 2018-2021 (1).xlsx"
df_1 = pd.read_excel(file_url)

In [92]:
df_1.to_sql(
    'tasks',
    con=engine,
    if_exists='replace',
    index=False
)

116879

In [93]:
df.to_sql(
    'employees_loc',
    con=engine,
    if_exists='replace',
    index=False
)

20000

In [9]:
database_file_path="./db/test.db"

In [10]:
print(f'sqlite:///{database_file_path}')

sqlite:///./Downloads/db/test.db


In [11]:
engine = create_engine(f'sqlite:///{database_file_path}')

In [12]:
from sqlalchemy import create_engine, inspect, text


# Connect to the database
with engine.connect() as connection:
    # Inspect the database
    inspector = inspect(connection)
    
    # Get table names
    tables = inspector.get_table_names()
    print("Tables in the database:", tables)

    # Loop through each table and print its content
    for table_name in tables:
        print(f"\nData in table '{table_name}':")
        query = text(f"SELECT * FROM {table_name} LIMIT 5;")  # Limit to 5 rows to avoid large outputs
        result = connection.execute(query)
        
        # Print column names
        print(result.keys())
        
        # Print rows
        for row in result:
            print(row)

Tables in the database: ['all_states_history', 'employees_loc', 'tasks']

Data in table 'all_states_history':
RMKeyView(['Employee ID', 'Name', 'Skill', 'Shift', 'Start Time', 'End Time', 'Gates_ID', 'LONGITUDE', 'LATITUDE'])
(1, 'Employee_1', 'Aircraft Painter', 'Morning', '06:00', '14:00', 'gate 007', -77.043, 38.904)
(2, 'Employee_2', 'Aircraft Painter', 'Night', '22:00', '06:00', 'gate 004', -77.032, 38.906)
(3, 'Employee_3', 'Aircraft Painter', 'Afternoon', '14:00', '22:00', 'gate 019', -77.023, 38.903)
(4, 'Employee_4', 'Aircraft Painter', 'Morning', '06:00', '14:00', 'gate 021', -77.028, 38.894)
(5, 'Employee_5', 'Metallic Structures Engineer', 'Night', '22:00', '06:00', 'gate 017', -77.024, 38.912)

Data in table 'employees_loc':
RMKeyView(['Employee ID', 'Name', 'Skill', 'Shift', 'Start Time', 'End Time', 'Gates_ID', 'LONGITUDE', 'LATITUDE'])
(1, 'Employee_1', 'Aircraft Painter', 'Morning', '06:00', '14:00', 'gate 007', -77.043, 38.904)
(2, 'Employee_2', 'Aircraft Painter', 'N

In [28]:
MSSQL_AGENT_PREFIX = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
## Tools:
"""

In [31]:
MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: Action: the action to take, can be one of [{tool_names}]
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: creating a table
Action Input: 
Columns:
Table : 'tasks'
Mxh EST.: Estimated labor-hours
Table : 'employees_loc'
Start Time: when the shift starts
End Time : when the shift ends

Observation:
(0.224), (2)
Thought:I now know the final answer
Final Answer: a table where you allocate employees to tasks where you used methods that saved time [indicate how much time is saved]

Explanation:
I used [indicate for example the method] that saves [indicate how much time is saved].
and I tried this method [indicate how much time it saves].
===> End of Example

"""

In [32]:
QUESTION = """allocate optimally the employees situated in the table employees_loc to the tasks situated in table tasks where you explain
the mathematical and modelization you did creating another table as a result
"""

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=model,
    toolkit=toolkit,
    top_k=30,
    verbose=True
    
    
)

In [33]:
agent_executor_SQL.invoke(QUESTION,handle_parsing_errors=True)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: all_states_history, employees_loc, tasksI need to analyze the schema of the tables "employees_loc" and "tasks" to understand their structure and columns.
Action: sql_db_schema
Action Input: employees_loc, tasks
CREATE TABLE employees_loc (
	"Employee ID" BIGINT, 
	"Name" TEXT, 
	"Skill" TEXT, 
	"Shift" TEXT, 
	"Start Time" TEXT, 
	"End Time" TEXT, 
	"Gates_ID" TEXT, 
	"LONGITUDE" FLOAT, 
	"LATITUDE" FLOAT
)

/*
3 rows from employees_loc table:
Employee ID	Name	Skill	Shift	Start Time	End Time	Gates_ID	LONGITUDE	LATITUDE
1	Employee_1	Aircraft Painter	Morning	06:00	14:00	gate 007	-77.043	38.904
2	Employee_2	Aircraft Painter	Night	22:00	06:00	gate 004	-77.032	38.906
3	Employee_3	Aircraft Painter	Afternoon	14:00	22:00	gate 019	-77.023	38.903
*/


CREATE TABLE tasks (
	"A/C" TEXT, 
	"ITEM" TEXT, 
	"DESCRIPTION" TEXT, 
	"BLOCK" TEXT, 
	"SKILL" TEXT, 
	"Mxh EST." FLOAT, 
	"PER FH" FLOAT, 
	"PER FC" FLOAT, 
	"

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: I have analyzed the schema of the tables "employees_loc" and "tasks" to understand their structure and columns. Now, I will create a query that optimally allocates employees to tasks and creates a new table as a result.

Action: SQL query
Action Input:
SELECT 
    e."Employee ID" AS "Employee ID",
    e.Name AS "Employee Name",
    t."A/C" AS "Aircraft",
    t.ITEM AS "Item",
    t.DESCRIPTION AS "Task Description",
    t.SKILL AS "Required Skill",
    t."Mxh EST." AS "Estimated Labor Hours"
FROM employees_loc e
JOIN tasks t ON e.Skill = t.SKILL
ORDER BY e."Employee ID" ASC
LIMIT 30;

Explanation: I created a query that selects the Employee ID, Employee Name, Aircraft, Item, Task Description, Required Skill, and Estimated Labo